In [1]:
# %load train.py
from models import *
from utils.utils import *
from utils.datasets import *
from utils.parse_config import *

import os
import sys
import time
import datetime
import argparse

import torch
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torch.autograd import Variable
import torch.optim as optim

from config import myolo_config
from mutils import *

In [2]:
from fastprogress import master_bar, progress_bar

In [3]:
import os 

In [4]:
opt=myolo_config("/media/tianning/DATA/COCO/data")

In [5]:
opt.batch_size=1
# Get data configuration
data_config = parse_data_config(opt.data_config_path)
train_path = opt.train
INIT_MODEL  =False
INIT_KERNEL =False
KERNEL_TRAIN=False
# Get dataloader
dataloader = torch.utils.data.DataLoader(ListDataset(train_path), batch_size=opt.batch_size, shuffle=False, num_workers=opt.n_cpu)
cuda = torch.cuda.is_available() and opt.use_cuda
classes = load_classes(opt.class_path)

model =MDarknet('config/m-yolov3.cfg')
#model =Darknet('config/yolov3.cfg')
#model =MDarknet('config/f-yolov3.cfg')
#model.load_weights('../PyTorch-YOLOv3-RPN/weights/yolov3.weights')

In [6]:
#torch.save(model.state_dict(),'origin.yolo.weights')

In [7]:
#state_dict=torch.load('origin.yolo.weights')

In [8]:
if INIT_MODEL:
    model.apply(weights_init_normal)
else:
    model.load_weights("checkpoints/latest.weights")
    #model.load_state_dict(torch.load('origin.fyolo.weights'))
if cuda:model = model.cuda()

In [9]:
#torch.save(model.state_dict(),'origin.yolo.weights')

# Train

### test

In [10]:
model.train();

In [11]:
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor
if INIT_KERNEL:
    cls_k_b_pool={}
    feed_cls_k_b=[generate_new_kernels(cls_k_b_pool,classes,1,1024,cuda),
              generate_new_kernels(cls_k_b_pool,classes,2,512,cuda),
              generate_new_kernels(cls_k_b_pool,classes,3,256,cuda)]
else:
    cls_k_b_pool=torch.load('checkpoints/start.plug_in.weights')
    feed_cls_k_b=review_plug_in(cls_k_b_pool,classes,cuda)

In [12]:
all_kernel_param=[x for l in feed_cls_k_b for kb in l for x in kb]
if KERNEL_TRAIN:
    for p in model.parameters():p.requires_grad=False
    for p in all_kernel_param: p.requires_grad =True
else:
    for p in model.parameters():p.requires_grad=True
    for p in all_kernel_param: p.requires_grad =True

In [13]:
# Get hyper parameters
hyperparams   = parse_model_config(opt.model_config_path)[0]
learning_rate = float(hyperparams["learning_rate"])
momentum      = float(hyperparams["momentum"])
decay         = float(hyperparams["decay"])
burn_in       = int(hyperparams["burn_in"])

lr=0.0001
# optimizer = torch.optim.Adam([
#                 {'params': filter(lambda p: p.requires_grad, model.parameters()),'lr': learning_rate},
#                  {'params': all_kernel_param, 'lr': learning_rate}
#             ])
optimizer = torch.optim.SGD([
                {'params': filter(lambda p: p.requires_grad, model.parameters())},
                 {'params': all_kernel_param}
            ],lr=lr, momentum=momentum, weight_decay = decay)

In [14]:
#from train_utils import *
losses = AverageMeter()
log_hand=RecordLoss([losses],[[]],100)
mb = master_bar(range(10))
mb.names = ['tloss']

In [15]:
for batch_i, (_, imgs, targets) in enumerate(dataloader):
        imgs = Variable(imgs.type(Tensor))
        targets = Variable(targets.type(Tensor), requires_grad=False)
        break

/home/tianning/.local/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


In [16]:
cls_k_b_pool['motorbike'].keys()

dict_keys(['b2', 'b3', 'b1', 'k2', 'k3', 'k1'])

### test inference

#### 开始检查weight,bais的对应关系

In [77]:
x80=model.get_layer_sin_output(imgs,80)
x81=model.get_layer_sin_output(imgs,81)

In [100]:
weight.shape

torch.Size([255, 1024, 1, 1])

In [10]:
key_name=['x','y','w','h','conf']+classes

In [11]:
store={}
for name in key_name:
    store[name]={}

In [8]:
for name,p in model.named_parameters():
    if 'conv_81' in name:
        if 'weight' in name:weight81=p
        if 'bias' in name:    bias81=p
        print(name)
    if 'conv_93' in name:
        if 'weight' in name:weight93=p
        if 'bias' in name:    bias93=p
        print(name)
    if 'conv_105' in name:
        if 'weight' in name:weight105=p
        if 'bias' in name:    bias105=p        
        print(name)

module_list.81.conv_81.weight
module_list.81.conv_81.bias
module_list.93.conv_93.weight
module_list.93.conv_93.bias
module_list.105.conv_105.weight
module_list.105.conv_105.bias


In [12]:
shift_weight=weight81.view(3,-1,1024,1,1).permute(1,0,2,3,4)
shift_bias  =bias81.view(3,-1).permute(1,0)
for name,k,b in zip(key_name,shift_weight,shift_bias):
    store[name]['k1'] =k
    store[name]['b1'] =b
shift_weight=weight93.view(3,-1,512,1,1).permute(1,0,2,3,4)
shift_bias  =bias93.view(3,-1).permute(1,0)
for name,k,b in zip(key_name,shift_weight,shift_bias):
    store[name]['k2'] =k
    store[name]['b2'] =b
shift_weight=weight105.view(3,-1,256,1,1).permute(1,0,2,3,4)
shift_bias  =bias105.view(3,-1).permute(1,0)
for name,k,b in zip(key_name,shift_weight,shift_bias):
    store[name]['k3'] =k
    store[name]['b3'] =b

In [13]:
torch.save(store,'coco_class_kernel.weight.bias')

In [57]:
x= model([imgs,feed_cls_k_b,80])[0]

In [60]:
F.conv2d(x,full_kernel,bias=full_bias).shape

torch.Size([1, 1200, 13, 13])

In [43]:
feed_cls_k_b[0][0][0].shape

torch.Size([15, 1024, 1, 1])

In [45]:
out = model([imgs,feed_cls_k_b,82])[0];out.shape

torch.Size([1, 80, 507, 5])

## test train

In [31]:
last=model.state_dict()['module_list.0.batch_norm_0.running_mean'].cpu().numpy()

In [34]:
now =model.state_dict()['module_list.0.batch_norm_0.running_mean'].cpu().numpy()
(now-last).sum()

0.0

In [30]:
model.train()
for module in model.modules():
    if isinstance(module, torch.nn.modules.BatchNorm1d):
        module.eval()
    if isinstance(module, torch.nn.modules.BatchNorm2d):
        module.eval()
    if isinstance(module, torch.nn.modules.BatchNorm3d):
        module.eval()  

BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)

BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)

BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)

BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)

BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)

...

BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)

BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)

BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)

BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)

BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)

In [33]:
steps=0
for epoch in mb:
    cur_lr = adjust_learning_rate(lr, optimizer, epoch, gamma=0.1)
    for batch_i, (_, imgs, targets) in enumerate(dataloader):
        imgs = Variable(imgs.type(Tensor))
        targets = Variable(targets.type(Tensor), requires_grad=False)
        loss = model([imgs,feed_cls_k_b], targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
      
        
        log_hand.step(steps,[loss])
        #log_hand.update_graph(mb,steps)
        print(
            "Epoch {}/{}, Batch {}/{} Losses:total {:.2f}, conf {:.2f},x {:.2f},y {:.2f},w {:.2f},h {:.2f}, r {:.1f}"
            .format(
                epoch,
                opt.epochs,
                batch_i,
                len(dataloader),
                loss.item(),
                model.losses["x"],
                model.losses["y"],
                model.losses["w"],
                model.losses["h"],
                model.losses["conf"],
                model.losses["recall"],
            ), end="\r")
        if np.isnan(loss.cpu() .detach().numpy()):sys.exit(0)
        model.seen += imgs.size(0)
        steps += 1
        if steps%10==1:
            log_hand.print2file(steps,os.path.join(opt.checkpoint_dir,"loss.log"))
    if epoch % opt.checkpoint_interval == 0:
            model.save_weights("%s/epoch_%d.weights" % (opt.checkpoint_dir, epoch))
            torch.save(cls_k_b_pool,"%s/epoch_%d.plug_in_80_class.weight"% (opt.checkpoint_dir, epoch))

/home/tianning/.local/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


KeyboardInterrupt: 

# Inference

#### Function

In [ ]:
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
def gray_print(img):
    plt.imshow(img,'gray')
def rgb_print(img):
    img_rgb=img[:,:,[2,1,0]]
    plt.imshow(img_rgb,'gray')
def non_max_suppression(prediction, conf_thres=0.5, nms_thres=0.4):
    """
    Removes detections with lower object confidence score than 'conf_thres' and performs
    Non-Maximum Suppression to further filter detections.
    Returns detections with shape:
        (x1, y1, x2, y2, object_conf, class_score, class_pred)
    """

    # From (center x, center y, width, height) to (x1, y1, x2, y2)
    box_corner = prediction.new(prediction.shape)
    box_corner[..., 0]  = prediction[..., 0] - prediction[..., 2] / 2
    box_corner[..., 1]  = prediction[..., 1] - prediction[..., 3] / 2
    box_corner[..., 2]  = prediction[..., 0] + prediction[..., 2] / 2
    box_corner[..., 3]  = prediction[..., 1] + prediction[..., 3] / 2
    prediction[..., :4] = box_corner[..., :4]

    batches,classes,boxes,_ = prediction.shape
    output = [None for _ in range(len(prediction))]
    for image_i, image_pred_class in enumerate(prediction):
        # Filter out confidence scores below threshold
        for c in range(classes):
            image_pred =image_pred_class[c] 
            # located at each class
            conf_mask  = (image_pred[:, 4] >= conf_thres).squeeze()
            image_pred = image_pred[conf_mask]
            # If none are remaining => process next image
            if not image_pred.size(0):continue
            # Detections ordered as (x1, y1, x2, y2, obj_conf, class_conf, class_pred)
            detections = image_pred
            # Iterate through all predicted classes
            # Get the detections with the particular class
            detections_class = detections
            # Sort the detections by maximum objectness confidence
            _, conf_sort_index = torch.sort(detections_class[:, 4], descending=True)
            detections_class = detections_class[conf_sort_index]
            # Perform non-maximum suppression
            max_detections = []
            while detections_class.size(0):
                # Get detection with highest confidence and save as max detection
                max_detections.append(detections_class[0].unsqueeze(0))
                # Stop if we're at the last detection
                if len(detections_class) == 1:break
                # Get the IOUs for all boxes with lower confidence
                ious = bbox_iou(max_detections[-1], detections_class[1:])
                # Remove detections with IoU >= NMS threshold
                detections_class = detections_class[1:][ious < nms_thres]

            max_detections = torch.cat(max_detections).data
            max_detections = F.pad(max_detections,(0,1,0,0),value=c)
            # Add max detections to outputs
            output[image_i] = (
                max_detections if output[image_i] is None else torch.cat((output[image_i], max_detections))
            )

    return output

In [ ]:
def naive_position(prediction):
    box_corner = prediction.new(prediction.shape)
    box_corner[..., 0]  = prediction[..., 0] - prediction[..., 2] / 2
    box_corner[..., 1]  = prediction[..., 1] - prediction[..., 3] / 2
    box_corner[..., 2]  = prediction[..., 0] + prediction[..., 2] / 2
    box_corner[..., 3]  = prediction[..., 1] + prediction[..., 3] / 2
    prediction[..., :4] = box_corner[..., :4]
    return prediction

#### Test

In [5]:
#model.train()
_=model.eval() # Set in evaluation mode

cls_k_b_pool=torch.load('checkpoints/latest.plug_in.weights')
feed_cls_k_b=review_plug_in(cls_k_b_pool,classes,cuda)

trn_path=train_path.replace('trainvalno5k','train5k')
val_path=train_path.replace('trainvalno5k','val5k')

opt.batch_size=10
dataloader = torch.utils.data.DataLoader(ListDataset(train_path), batch_size=opt.batch_size, shuffle=False, num_workers=opt.n_cpu)
classes = load_classes(opt.class_path) # Extracts class labels from file
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

In [7]:
for batch_i, (img_paths, input_imgs, targets) in enumerate(dataloader):
    img_path =img_paths
    input_img=input_imgs
    target =targets.cpu().numpy().reshape(-1,5)
    #print(targets)
    break

In [8]:
#Get detections
with torch.no_grad():
    input_imgs = Variable(input_img.type(Tensor))
    test = model([input_imgs,feed_cls_k_b,81])

In [9]:
#Get detections
with torch.no_grad():
    input_imgs = Variable(input_img.type(Tensor))
    detections = model([input_imgs,feed_cls_k_b])
#detections = non_max_suppression(detections, 0.95, 0.5)

In [10]:
detection=naive_position(detections[0]);detection.shape

NameError: name 'naive_position' is not defined

In [ ]:
for i,class_pred in enumerate(detection):
    if torch.max(class_pred[:, 4]) > 0.9:
        print(classes[i])

In [ ]:
torch.max(detection[16][:,4])

In [ ]:
classes.index('dog')

In [ ]:
layer.bce_loss(pred_conf, tconf)

In [ ]:
image_pred = detection
conf_mask  = (image_pred[:, 4] >= 0.9 ).squeeze()
image_pred = image_pred[conf_mask]

In [ ]:
rgb_print(img)

In [ ]:
ll=draw_box_with_label(img_flow,[tdet])
rgb_print(ll)

In [ ]:
def back2origin(x1,y1,x2,y2,pad_x,pad_y,gain,scale):
    x1-= pad_x
    x2-= pad_x
    y1-= pad_y
    y2-= pad_y
    x1/= gain
    x2/= gain
    y1/= gain
    y2/= gain
    c_x,c_y,w,h=(x1+x2)/2,(y1+y2)/2,(x2-x1),(y2-y1)
    w*=scale
    h*=scale
    x1,y1,x2,y2=c_x-w/2,c_y-h/2,c_x+w/2,c_y+h/2
    return int(x1),int(y1),int(x2),int(y2)
def draw_box_with_label(img_origin,detections,input_img_size=416.0):
    img=img_origin.copy()
    img0_shape=img.shape
    img_size=416
    scale=1.0*img_size/input_img_size
    print(scale)
    gain = float(img_size) / max(img0_shape)  # gain  = old / new
    pad_x = (img_size - img0_shape[1] * gain) / 2  # width padding
    pad_y = (img_size - img0_shape[0] * gain) / 2  # height padding
    
    # Draw bounding boxes and labels of detections
    if detections is not None:
        detections    = detections[0]
        unique_labels = detections[:, -1]
        n_cls_preds   = len(unique_labels)
        bbox_colors   = (0,255,0)
        for x1, y1, x2, y2, cls_pred,cls_conf in detections:
            conf=cls_conf
            x1,y1,x2,y2=back2origin(x1,y1,x2,y2,pad_x,pad_y,gain,scale)
            color =(0,255,0)
            _=cv2.rectangle(img, (x1,y1), (x2, y2), color, 2)  
            _=cv2.putText(img, classes[int(cls_pred)], (x1,y1), cv2.FONT_HERSHEY_COMPLEX, 1.2, (255, 255, 255), 2)
    return img